In [1]:
import numpy as np
import pandas as pd
import string
import re
import functools

In [2]:
from google.colab import files
uploaded = files.upload()

Saving input.txt to input (1).txt


In [3]:
with open("input.txt", 'r') as f:
    inputs = f.readlines()

In [ ]:
# ===============================
# Part A
# ===============================

In [4]:
split_games = [i.split(": ") for i in inputs]
split_draws = [(i[0], i[1].strip().split('; ')) for i in split_games]
parsed_games = [(i[0], [j.split(", ") for j in i[1]]) for i in split_draws]

In [5]:
def merge_dictionaries(dict1, dict2):
    merged_dict = dict1.copy()
    merged_dict.update(dict2)
    return merged_dict

def convert_to_dict(xs: list[str]) -> dict[str, int]:
    together = functools.reduce(merge_dictionaries, [{i.split(" ")[1]: int(i.split(" ")[0])} for i in xs])
    return together

In [27]:
game_list = []
for game in parsed_games:
    df = (
        pd.DataFrame(list({f"draw {index}": convert_to_dict(draw) for (index, draw) in enumerate(game[1])}.values()))
        .assign(Game = game[0])
        .reset_index().
        rename(columns = {"index": 'draw'})
    )
    game_list.append(df)

df = pd.concat(game_list).fillna(0).assign(game_number = lambda x: x.Game.str.rsplit(pat = " ", n = 1).map(lambda y: int(y[1])))

In [48]:
invalid_draws = df[lambda x: (x.red > 12) | (x.green > 13) | (x.blue > 14)]
invalid_games = invalid_draws.game_number.unique()
valid_games = df[lambda x: ~x.game_number.isin(invalid_games)]

In [54]:
sum(valid_games.game_number.unique())

2207

In [ ]:
# ===============================
# Part B
# ===============================

In [58]:
(max_values := df.groupby("game_number")[['red', 'green', 'blue']].max())

,red,green,blue
game_number,,,
1,15.0,18.0,6.0
2,10.0,13.0,1.0
3,2.0,7.0,1.0
4,5.0,9.0,14.0
5,5.0,12.0,12.0
...,...,...,...
96,9.0,1.0,10.0
97,11.0,3.0,1.0
98,13.0,13.0,4.0


In [61]:
max_values.apply(lambda x: x['red'] * x['green'] * x['blue'], axis = 1).sum()

62241.0